In [ ]:
run_times = ['09-02-2025_01_19_00', '09-02-2025_03_05_57']

#run_times = ['10-02-2025_00_20_55', '10-02-2025_02_16_16']

## Drawing plots

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker

# LaTeX configuration and style setup
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.size": 8,
    "axes.titlesize": 8,
    "legend.fontsize": 8,
    "axes.labelsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "lines.linewidth": 0.8,
    "axes.linewidth": 0.5,
    "axes.edgecolor": 'gray'
})

# Document dimensions
cm = 1/2.54  # centimeters in inches
text_width = 17 * cm

# Plot parameters
FIG_SIZE = (text_width, 0.5 * text_width * 0.8)  # Golden ratio
SHOW_LEGEND = True  #Set this to True when needed

def create_subplot(position, run_time, ax):
    path = 'out/run_' + run_time + '/'
    with open(path + 'run_info.json', 'r') as file:
        run_info = json.load(file)

    max_y = 0
    for idx, algorithm in enumerate(run_info['algorithms']):
        algorithm_underscored = algorithm.replace(' ', '_')

        points = pd.read_csv(path + algorithm_underscored + '_points.csv')
        bound = pd.read_csv(path + algorithm_underscored + '_bound.csv')

        k_values = np.sort(points['k'].unique())
        sqrt_values = np.sqrt(points['value'])

        mean_values = sqrt_values.groupby(points['k']).mean()
        std_values = sqrt_values.groupby(points['k']).std()
        ci = std_values

        color = plt.cm.tab10(idx)

        #ax.scatter(points['k'], np.sqrt(points['value']), color=color, alpha=0.1, s=1)

        ####

        # Main plot with confidence interval
        ax.set_xlim(k_values[0], k_values[0] + (k_values[-1] - k_values[0]) * (51.0 / 50))
        max_y = max(max_y, np.max(sqrt_values))
        ax.set_ylim(0, max_y * (51.0 / 50))
        ax.plot(k_values, mean_values, color=color)
        ax.fill_between(k_values, mean_values - ci, mean_values + ci,
                        color=color, alpha=0.3, linewidth=0)


        # Theoretical bound
        if not (bound['value'] == 0).all():
            ax.plot(bound['k'], np.sqrt(bound['value']),
                    color=color, linestyle='--')
    # Manage tiks
    ax.minorticks_on()
    ax.tick_params(axis='both', which='major', length=2, width=0.5, color='gray', direction='in')
    ax.tick_params(axis='both', which='minor', length=1, width=0.5, color='gray', direction='in')


    # Axis labels
    ax.set_xlabel(r'$k$')
    if position == 'left':
        ax.set_ylabel(r'$\Vert X^\dag \Vert_2 / \Vert X_\mathcal{S}^\dag \Vert_2$')

    # Grid and borders
    ax.set_axisbelow(True)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)


def create_plot(run_times):
    fig, ax = plt.subplots(figsize=FIG_SIZE, nrows=1, ncols=2, layout='constrained')

    create_subplot('left', run_times[0], ax[0])
    create_subplot('right', run_times[1], ax[1])

    path = 'out/run_' + run_times[0] + '/'
    with open(path + 'run_info.json', 'r') as file:
        run_info = json.load(file)

    # Create custom legend
    if SHOW_LEGEND:
        custom_legend = []
        for algorithm in run_info['algorithms']:
            color = plt.cm.tab10(list(run_info['algorithms']).index(algorithm))
            custom_legend.append(plt.Line2D([0], [0], marker='s', color='w', markerfacecolor=color, markeredgecolor='white', label=algorithm, markersize=6))

        custom_legend.append(plt.Line2D([0], [0], linestyle='-', label='mean value', color='black'))
        custom_legend.append(plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markeredgecolor='white', alpha=0.3, label='standard deviation', markersize=6))
        custom_legend.append(plt.Line2D([0], [0], linestyle='--', label='theoretical bound', color='black'))

        leg = fig.legend(handles=custom_legend, framealpha=0.9,
                       loc='upper center', bbox_to_anchor=(0.5, 0.00), fancybox=True, ncols=4)
        #leg.get_frame().set_linewidth(0.5)
        leg.get_frame().set_edgecolor("1")

    plt.savefig('plot.pdf', bbox_inches='tight')
    plt.show()
    plt.close()

create_plot(run_times)